In [ ]:
import pandas as pd
import numpy as np

#Carga del dataset

startDataset = pd.read_csv('books.csv', on_bad_lines= 'skip')

startDataset.head()

In [ ]:
# Modificacion de columnas

# Eliminacion de columnas
newDataset = startDataset.drop(["bookID", "average_rating", "isbn", "isbn13", "language_code", "ratings_count", "text_reviews_count"], axis = 1)

# Visualizacion del tipo de dato "publication_date"
print("\n\nAntes")
print(newDataset.dtypes)

# Si encuentra un error en el formato de la fecha elimina la fecha
print("\n\nAhora")
newDataset["publication_date"] = pd.to_datetime(newDataset["publication_date"], format = '%m/%d/%Y', errors = 'coerce')
print(newDataset.dtypes)

# Modificacion de la columna "publication_date" al formato YYYY
newDataset["publication_date"] = newDataset["publication_date"].dt.strftime('%Y')

# Si hubo un error en publication_date va a eliminar la fila si esta vacia
newDataset = newDataset.drop(newDataset[newDataset["publication_date"] == "NaN"].index)

newDataset.head()

In [ ]:
# Creacion de tablas de libros para su uso en la base de datos

# Dataframe libro
dfLibro = newDataset
# Dataframe autor
dfAutor = pd.DataFrame(columns = ["idAutor", "nombresApellidos"])

# Dataframe editorial
dfEditorial = pd.DataFrame(columns = ["idEditorial", "nombre"])

dfLibro.head()
dfAutor.head()
dfEditorial.head()

In [ ]:
# Ingreso de datos en las tabla autor
dfAutor = dfAutor.set_index("idAutor")
dfAutor["nombresApellidos"] = newDataset["authors"]
dfAutor.drop_duplicates(subset=["nombresApellidos"], inplace=True)
dfAutor = dfAutor.reset_index(drop=True)
dfAutor.index = dfAutor.index + 1
dfAutor["idAutor"] = dfAutor.index
dfAutor = dfAutor[["idAutor", "nombresApellidos"]]

dfAutor.head()

In [ ]:
# Ingreso de datos en las tabla editorial

dfEditorial = dfEditorial.set_index("idEditorial")
dfEditorial["nombre"] = newDataset["publisher"]
dfEditorial.drop_duplicates(subset=["nombre"], inplace=True)
dfEditorial = dfEditorial.reset_index(drop=True)
dfEditorial.index = dfEditorial.index + 1
dfEditorial["idEditorial"] = dfEditorial.index
dfEditorial = dfEditorial[["idEditorial", "nombre"]]

dfEditorial.head()

In [ ]:
# Modificacion de la tabla libros para su separacion por authores y editoriales
dfLibro = pd.merge(dfLibro, dfAutor, left_on="authors", right_on="nombresApellidos")
dfLibro = pd.merge(dfLibro, dfEditorial, left_on = "publisher", right_on = "nombre")

dfLibro = dfLibro.drop(["authors", "nombresApellidos", "publisher", "nombre"],axis = 1)

dfLibro.index.name = "idLibro"
dfLibro.index = dfLibro.index + 1

# Columna precio
dfLibro["precio"] = np.random.randint(3000, 10000, dfLibro.shape[0])/100
dfLibro = dfLibro.rename(columns={"title":'nombre', "publication_date":"anhoPublicado", "num_pages": "numPaginas"})
dfLibro.columns = ['nombre', 'numPaginas', 'anhoPublicado', 'idAutor', 'idEditorial', 'precio']

dfLibro.head(50)

In [ ]:
# Guardado de Archivos


dfLibro.to_csv("tablaLibro.csv", columns=["idEditorial", "idAutor", "nombre", "precio", "numPaginas", "anhoPublicado"])
dfAutor.to_csv("tablaAutor.csv", index=False)
dfEditorial.to_csv("tablaEditorial.csv", index=False)